asd

In [1]:
!pip install datasets model2vec reach numpy tqdm 
from datasets import load_dataset
from model2vec import StaticModel
from reach import Reach
import numpy as np
from tqdm import tqdm
from time import perf_counter

/Users/thomasvandongen/Recommenders/repositories/recommenders-shared-tools/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


asdasd

In [11]:
# Load the model and dataset
model = StaticModel.from_pretrained("minishlab/M2V_base_output")
ds = load_dataset("ag_news")["train"]
texts = ds['text']

In [13]:
#texts = ["asd", "asd", "tads", "qwea"]
seen = set()
deduplicated_text_indices = np.array([i for i, text in enumerate(texts) if text not in seen and not seen.add(text)])
len(deduplicated_text_indices)

120000

In [4]:
# Encode texts into embeddings
embeddings = model.encode(texts, show_progressbar=True)
embedding_matrix = np.vstack(embeddings)

100%|██████████| 118/118 [00:02<00:00, 47.80it/s]


In [ ]:
def deduplicate(embedding_matrix: np.ndarray, threshold: float, batch_size: int = 1024) -> np.ndarray:
    """
    Deduplicate embeddings and return the deduplicated indices.
    
    :param embedding_matrix: The embeddings to deduplicate.
    :param threshold: The similarity threshold to use for deduplication.
    :param batch_size: The batch size to use for similarity computation.
    :return: The deduplicated indices.
    """
    reach = Reach(vectors=embedding_matrix, items=[str(i) for i in range(len(embedding_matrix))])
    
    # Find similar documents
    similarity_cutoff = 1 - threshold
    is_duplicate = np.zeros(len(embedding_matrix), dtype=bool)
    
    results = reach.threshold(
        [str(i) for i in range(len(embedding_matrix))], 
        threshold=similarity_cutoff, 
        batch_size=batch_size, 
        show_progressbar=True
    )
    
    # Process duplicates
    for i, similar_items in tqdm(enumerate(results), total=len(results)):
        if is_duplicate[i]:
            continue  # Skip already marked duplicates

        similar_indices = [int(item[0]) for item in similar_items if int(item[0]) != i]
        is_duplicate[similar_indices] = True

    deduplicated_indices = np.where(~is_duplicate)[0]

    return deduplicated_indices

In [ ]:
deduplicated_indices = deduplicate(embedding_matrix, threshold=0.9)
# Get deduplicated documents and embeddings
deduplicated_docs = [texts[i] for i in deduplicated_indices]
deduplicated_embeddings = embedding_matrix[deduplicated_indices]